In [141]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from azure.storage.blob import BlobServiceClient

subscription_id = '96599b9b-0d9b-4577-9163-418d2c6cd411'
resource_group = 'rg-datasci-ml-dev-001'
workspace_name = 'ml-moss-dev-001'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_hourly_compare_cmic = Dataset.get_by_name(workspace, name='Hourly_Employee_Time_prod')
cmic_pr_df_dev = dataset_hourly_compare_cmic.to_pandas_dataframe()

dataset_hourly_compare_wd = Dataset.get_by_name(workspace, name='WD_Hourly_Employee_Time_prod')
wd_pr_df_dev = dataset_hourly_compare_wd.to_pandas_dataframe()

tcp_dataset = Dataset.get_by_name(workspace, name='WorkSegments_Timesheet_prod')
tcp_df_dev = tcp_dataset.to_pandas_dataframe()

jcjob_dataset = Dataset.get_by_name(workspace, name='JCJOB_TABLE_TN_prod')
jcjob_df = jcjob_dataset.to_pandas_dataframe()

In [ ]:
#Filter out EEIDs sent via email - Need list from Ileana
#Add EE Name to the excel file



In [142]:
##########################

week_ending = '2023-10-14'

##########################

date = datetime.strptime(week_ending, "%Y-%m-%d")
start_date = date - timedelta(days = 6)
end_date = date

In [163]:
tcp_df = tcp_df_dev.copy()

In [153]:
###TCP Data Manipulation
#Filter: Solar Jobs only 
tcp_df = tcp_df.loc[tcp_df['Job_Code'].str.startswith('710')]
#Filter: Only Keeping Labor Hourly
tcp_df = tcp_df.loc[tcp_df['Task'].isin(['LH','LHA'])]
#Filter: Only keeping Moss Labor Hourly - excluding subs
tcp_df = tcp_df.loc[tcp_df['Department']=='710']
#Excluding this list of employees because they submit their time directly into Workday
tcp_df = tcp_df.loc[tcp_df['Employee_Number'].notin()]
#Creating a total hours column
tcp_df['total_hours'] = tcp_df['Regular_Hours'] + tcp_df['Overtime_Hours']
#Renaming Relevant Columns
tcp_df = tcp_df.rename({'Employee_Number':'EEID','Job_Code':'Job','Activity':'Sub_Job','total_hours':'Hours'},axis='columns')
tcp_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
tcp_df_comp = tcp_df.loc[:,tcp_cols_keep]

In [164]:
tcp_df

,WorkSegments_Timesheet_ID,Employee_Number,First_Name,Last_Name,Date,Job_Code,Activity,Phase_Code,Task,Regular_Hours,Overtime_Hours,Pay_Rate,Employee_Classification,Department,FileName,Created_Date
0,1,10000,ABRAHAN,SERNA,2022-06-18,7102106,15,48.9908.00,LH,8.0,0.0,18.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
1,2,10000,ABRAHAN,SERNA,2022-06-08,7102106,15,48.9908.00,LH,10.0,0.0,18.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
2,3,10000,ABRAHAN,SERNA,2022-06-16,7102106,15,48.9908.00,LH,10.0,0.0,18.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
3,4,10000,ABRAHAN,SERNA,2022-06-07,7102106,15,48.9908.00,LH,10.0,0.0,18.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
4,5,10000,ABRAHAN,SERNA,2022-06-20,7102106,15,48.9908.00,LH,10.0,0.0,18.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831718,831719,9999,ROLANDO,WASHINGTON,2022-08-20,7102106,25,48.7005.00,LH,8.0,0.0,18.5,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
831719,831720,9999,ROLANDO,WASHINGTON,2022-09-16,7102106,25,48.7005.00,LH,2.5,0.0,18.5,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
831720,831721,9999,ROLANDO,WASHINGTON,2022-09-26,7102106,25,48.7005.00,LH,10.0,0.0,18.5,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570
831721,831722,9999,ROLANDO,WASHINGTON,2022-09-29,7102106,25,48.7005.00,LH,10.0,0.0,18.5,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-17 09:31:20.570


In [14]:
wd_pr_df = wd_pr_df_dev.copy()

In [15]:
#Workday Data Manipulation
#Drop Null values if the employee does not have a job/sub job assignment
wd_pr_df = wd_pr_df.loc[~wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].isna()]
#Filter: Solar Jobs Only
wd_pr_df = wd_pr_df.loc[wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.startswith('710')]
#Extracting Employee ID (EEID) from 'Workers' column
wd_pr_df['EEID'] = wd_pr_df['Workers'].str.split('(').str[1]
wd_pr_df['EEID'] = wd_pr_df['EEID'].str.split(')').str[0]
#This isolates the 2 digit sub job, also accomodates the sub jobs that have a letter in them like Claims (C) and Warranty (W)
wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.replace('C' , '.C')
wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.replace('W' , '.W')
wd_pr_df['sub_job'] = wd_pr_df['cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str.split('.').str[1]
wd_pr_df['sub_job'] = wd_pr_df['sub_job'].str.split(' ').str[0]
#Leaves behind Job ONLY
wd_pr_df['job'] = wd_pr_df.loc[:,'cf_INTLRVCalculatedProjectFromTimeBlock.descriptor'].str[0:7]
#Renaming Relevant Columns
wd_pr_df = wd_pr_df.rename({'calculatedDate':'Date','job':'Job','sub_job':'Sub_Job','amount':'Hours'},axis='columns')
wd_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
wd_pr_df_comp = wd_pr_df.loc[:,wd_cols_keep]

In [16]:
cmic_pr_df = cmic_pr_df_dev.copy()

In [17]:
#CMiC Data Manipulation
#Re-casting this column as a Date data type
cmic_pr_df["Date"] = pd.to_datetime(cmic_pr_df["Date"])
#Filter: Only Keeping Labor Hourly
cmic_pr_df = cmic_pr_df.loc[cmic_pr_df['Task'].isin(['LH','LHA'])]
#Renaming Relevant Columns
cmic_pr_df = cmic_pr_df.rename({'Employee Number':'EEID','Job Code':'Job','Sub Job Code':'Sub_Job','Regular Hours':'Hours'},axis='columns')
cmic_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
cmic_pr_df_comp = cmic_pr_df.loc[:,cmic_cols_keep]

In [18]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

#### Grouping the Data by Different Levels

In [19]:
#Grouping by Job
tcp_grouped_job = tcp_df_comp.loc[tcp_filter].groupby(['Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_job = tcp_grouped_job.reset_index()
cmic_grouped_job = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_job = cmic_grouped_job.reset_index()
wd_grouped_job = wd_pr_df_comp.loc[wd_filter].groupby(['Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_job = wd_grouped_job.reset_index()

In [20]:
#Grouping by Job, Sub Job
tcp_grouped_subjob = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_subjob = tcp_grouped_subjob.reset_index()
cmic_grouped_subjob = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_subjob = cmic_grouped_subjob.reset_index()
wd_grouped_subjob = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_subjob = wd_grouped_subjob.reset_index()

In [21]:
#Grouping by Job, Sub Job, EEID
tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_grouped_ee.reset_index()
cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_grouped_ee.reset_index()
wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_grouped_ee.reset_index()

#### Unique Column Creation for the Joins

In [22]:
#Unique Columns to Join On for Sub Job Comparison
tcp_grouped_subjob['unique'] = tcp_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
cmic_grouped_subjob['unique'] = cmic_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
wd_grouped_subjob['unique'] = wd_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

#### TCP vs. Workday Comparison

In [23]:
#TCP v. WD Job
tcp_v_wd_job = tcp_grouped_job.merge(wd_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_job['TotalHours_WD'] = tcp_v_wd_job['TotalHours_WD'].fillna(0)
tcp_v_wd_job['TotalHours_TCP'] = tcp_v_wd_job['TotalHours_TCP'].fillna(0)
tcp_v_wd_job['Hours_Variance (TCP - WD)'] = tcp_v_wd_job['TotalHours_TCP'] - tcp_v_wd_job['TotalHours_WD']
tcp_v_wd_job = tcp_v_wd_job.sort_values(['Job'  , 'Hours_Variance (TCP - WD)'] , ascending=[False,False])

In [24]:
#TCP v. WD Job, Sub Job
tcp_v_wd_subjob = tcp_grouped_subjob.merge(wd_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_wd_subjob['TotalHours_WD'] = tcp_v_wd_subjob['TotalHours_WD'].fillna(0)
tcp_v_wd_subjob['TotalHours_TCP'] = tcp_v_wd_subjob['TotalHours_TCP'].fillna(0)
tcp_v_wd_subjob['Hours_Variance (TCP - WD)'] = tcp_v_wd_subjob['TotalHours_TCP'] - tcp_v_wd_subjob['TotalHours_WD']
tcp_v_wd_subjob = tcp_v_wd_subjob.sort_values(['Job_TCP' , 'Sub_Job_TCP' , 'Hours_Variance (TCP - WD)'] , ascending=[False,True,False])

In [25]:
#TCP v. WD Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_ee['TotalHours_WD'] = tcp_v_wd_ee['TotalHours_WD'].fillna(0)
tcp_v_wd_ee['TotalHours_TCP'] = tcp_v_wd_ee['TotalHours_TCP'].fillna(0)
tcp_v_wd_ee['Hours_Variance (TCP - WD)'] = tcp_v_wd_ee['TotalHours_TCP'] - tcp_v_wd_ee['TotalHours_WD']
tcp_v_wd_ee = tcp_v_wd_ee.sort_values('Hours_Variance (TCP - WD)' , ascending=False, key=abs)
#Removing Variances that are 0 - excluding EEIDs that match
tcp_v_wd_ee = tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0.0]

#### TCP vs. CMiC Comparison

In [26]:
#TCP v CMiC Job
tcp_v_cmic_job = tcp_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_job['TotalHours_CMiC'] = tcp_v_cmic_job['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_job['TotalHours_TCP'] = tcp_v_cmic_job['TotalHours_TCP'].fillna(0)
tcp_v_cmic_job['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_job['TotalHours_TCP'] - tcp_v_cmic_job['TotalHours_CMiC']
tcp_v_cmic_job = tcp_v_cmic_job.sort_values(['Job' , 'Hours_Variance (TCP - CMiC)'], ascending=[False, False])

In [27]:
#TCP v CMiC Job, Sub Job
tcp_v_cmic_subjob = tcp_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_cmic_subjob['TotalHours_CMiC'] = tcp_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_subjob['TotalHours_TCP'] = tcp_v_cmic_subjob['TotalHours_TCP'].fillna(0)
tcp_v_cmic_subjob['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_subjob['TotalHours_TCP'] - tcp_v_cmic_subjob['TotalHours_CMiC']
tcp_v_cmic_subjob = tcp_v_cmic_subjob.sort_values(['Job' , 'Sub_Job', 'Hours_Variance (TCP - CMiC)'], ascending=[False, True, False])

In [28]:
#TCP v CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_ee['TotalHours_CMiC'] = tcp_v_cmic_ee['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_ee['TotalHours_TCP'] = tcp_v_cmic_ee['TotalHours_TCP'].fillna(0)
tcp_v_cmic_ee['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_ee['TotalHours_TCP'] - tcp_v_cmic_ee['TotalHours_CMiC']
tcp_v_cmic_ee = tcp_v_cmic_ee.sort_values('Hours_Variance (TCP - CMiC)', ascending=False, key=abs)
#Filter down to relevant columns only

#### Workday vs. CMiC Comparison

In [29]:
#WD v. CMiC EEID
wd_v_cmic_job = wd_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_job['TotalHours_CMiC'] = wd_v_cmic_job['TotalHours_CMiC'].fillna(0)
wd_v_cmic_job['TotalHours_WD'] = wd_v_cmic_job['TotalHours_WD'].fillna(0)
wd_v_cmic_job['Hours_Variance (WD - CMiC)'] = wd_v_cmic_job['TotalHours_WD'] - wd_v_cmic_job['TotalHours_CMiC']
wd_v_cmic_job = wd_v_cmic_job.sort_values(['Job' , 'Hours_Variance (WD - CMiC)'], ascending=[False,False])

In [30]:
#WD v. CMiC Job, Sub Job
wd_v_cmic_subjob = wd_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
wd_v_cmic_subjob['TotalHours_CMiC'] = wd_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
wd_v_cmic_subjob['TotalHours_WD'] = wd_v_cmic_subjob['TotalHours_WD'].fillna(0)
wd_v_cmic_subjob['Hours_Variance (WD - CMiC)'] = wd_v_cmic_subjob['TotalHours_WD'] - wd_v_cmic_subjob['TotalHours_CMiC']
wd_v_cmic_subjob = wd_v_cmic_subjob.sort_values(['Job_WD' , 'Sub_Job_WD' , 'Hours_Variance (WD - CMiC)'], ascending=[False,True,False])

In [31]:
#WD v. CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_ee['TotalHours_CMiC'] = wd_v_cmic_ee['TotalHours_CMiC'].fillna(0)
wd_v_cmic_ee['TotalHours_WD'] = wd_v_cmic_ee['TotalHours_WD'].fillna(0)
wd_v_cmic_ee['Hours_Variance (WD - CMiC)'] = wd_v_cmic_ee['TotalHours_WD'] - wd_v_cmic_ee['TotalHours_CMiC']
wd_v_cmic_ee = wd_v_cmic_ee.sort_values('Hours_Variance (WD - CMiC)', ascending=False , key=abs)
#Filter down to relevant columns only

In [32]:
file_path = os.getcwd() + '/excel_output'
file_path

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jbeachy2/code/Users/JBeachy/Project_DataDomain/Payroll/git_LaborHours_Compare/excel_output'

In [33]:
#File Path in AMLS
file_name = f'Hours_Comparison_WeekEnding_{week_ending}'
#Writing to AMLS File Directory
with pd.ExcelWriter(f'{file_path}/{file_name}.xlsx') as writer:
    tcp_v_wd_job.to_excel(writer , sheet_name = 'TCPvWD_Hours_byJob' , index = False)
    tcp_v_wd_subjob.to_excel(writer , sheet_name = 'TCPvWD_Hours_bySubJob' , index = False)
    tcp_v_wd_ee.to_excel(writer , sheet_name = 'TCPvWD_Hours_byEE' , index = False)
    tcp_v_cmic_job.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byJob' , index = False)
    tcp_v_cmic_subjob.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_bySubJob' , index = False)
    tcp_v_cmic_ee.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byEE' , index = False)
    wd_v_cmic_job.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byJob' , index = False)
    wd_v_cmic_subjob.to_excel(writer , sheet_name = 'WDvCMiC_Hours_bySubJob' , index = False)
    wd_v_cmic_ee.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byEE' , index = False)

In [34]:
#Blob Storage Info
storage_account_key = 'u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA=='
storage_account_name = 'mossdatalakesource'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=mossdatalakesource;AccountKey=u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA==;EndpointSuffix=core.windows.net'
container_name = 'cmic'
#Defininng Blob Storage Uplpoad#
def uploadtoblobstorage(file_path,file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container = container_name , blob = file_name)

    with open(file_path , 'rb') as data:
        blob_client.upload_blob(data)
    print(f'Uploaded {file_name}.')

In [35]:
file_name_exp = f'{file_name}.xlsx'
uploadtoblobstorage(f'{file_path}/{file_name}.xlsx', file_name_exp)

Uploaded Hours_Comparison_WeekEnding_2023-10-14.xlsx.


#### Run Statistics 

In [104]:
#### TCP vs. Workday Stats
today = date.today()
today_timechange = today - timedelta(hours=4)
today_for_text = today_timechange.strftime("%m-%d-%Y %H:%M:%S")
print("TCP vs. Workday")
print("Count of EEIDs that are showing Zero Hours Worked in Workday: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_WD'] == 0]))
print("Count of EEIDs that are showing Zero Hours Worked in TCP: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_TCP'] == 0]))
print("Count of EEIDs with a variance less than 10 hours(ABS), not including 0: ", len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']< 10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] >-10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0)]))
print("Count of EEIDs with no variance in Hours Worked: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] == 0]))
print("Count of EEIDs with variance greater than or equal to 10(ABS): ",len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']>=10)|(tcp_v_wd_ee['Hours_Variance (TCP - WD)']<=-10)]))
print(f"Ran on {today_for_text} for Week Ending {week_ending}")

TCP vs. Workday
Count of EEIDs that are showing Zero Hours Worked in Workday:  254
Count of EEIDs that are showing Zero Hours Worked in TCP:  13
Count of EEIDs with a variance less than 10 hours(ABS), not including 0:  6
Count of EEIDs with no variance in Hours Worked:  2562
Count of EEIDs with variance greater than or equal to 10(ABS):  258
Ran on 10-17-2023 10:33:33 for Week Ending 2023-10-14
